# FreqAI GPU Backtest - Remote Setup

✅ **100% FREE** - No registration, no token, no credit card!

**Steps:**
1. Enable GPU: Runtime > Change runtime type > GPU (T4)
2. Run all cells: Runtime > Run all
3. Copy connection details
4. Run backtest from your local machine

In [ ]:
# 1. Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv || echo '❌ GPU not found. Enable GPU!'

In [ ]:
%%capture
# 2. Install SSH
!apt-get update -qq
!apt-get install -y openssh-server

In [ ]:
# 3. Setup SSH
import secrets, string
password = ''.join(secrets.choice(string.ascii_letters + string.digits) for _ in range(16))
!mkdir -p /var/run/sshd
!echo "root:{password}" | chpasswd
!sed -i 's/#PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config
!sed -i 's/#PasswordAuthentication yes/PasswordAuthentication yes/' /etc/ssh/sshd_config
!service ssh restart
print(f'✅ SSH ready | Password: {password}')

In [ ]:
# 4. Start tunnel with localhost.run (SSH-based, 100% free!)
import subprocess, time, re
print('⏳ Starting localhost.run tunnel...')
print('This uses SSH reverse tunneling - no installation needed!')
print()

# Start SSH reverse tunnel to localhost.run
proc = subprocess.Popen(
    ['ssh', '-R', '80:localhost:22', 'nokey@localhost.run'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    stdin=subprocess.PIPE,
    universal_newlines=True
)

# Wait for tunnel URL in output
tunnel_url = None
tunnel_port = None
for _ in range(30):
    line = proc.stdout.readline()
    if not line:
        time.sleep(1)
        continue
    
    print(line.strip())
    
    # Look for URL pattern: Connect via SSH to this URL: xxx.lhr.life:12345
    # Or: xxx.localhost.run port 12345
    match = re.search(r'([a-z0-9.-]+):(\d+)', line)
    if match and 'localhost.run' in line or 'lhr.life' in line:
        tunnel_url = match.group(1)
        tunnel_port = match.group(2)
        break

if tunnel_url and tunnel_port:
    full_url = f"{tunnel_url}:{tunnel_port}"
    print('\n' + '='*70)
    print('✅ TUNNEL ACTIVE')
    print('='*70)
    print(f'URL: {full_url}')
    print(f'Password: {password}')
    print('='*70)
    print('\n📋 Run on LOCAL machine:\n')
    print(f'python tools/backtest_executor.py \\')
    print(f'  --tunnel-url "{full_url}" \\')
    print(f'  --password "{password}" \\')
    print(f'  --strategy FreqAIHybridStrategy \\')
    print(f'  --timerange 20250901-20251012 \\')
    print(f'  --pairs BTC/USDT:USDT')
    print('\n' + '='*70)
else:
    print('\n⚠️  Could not detect tunnel URL automatically')
    print('Check the output above for connection details')
    print('\nTunnel is still running - look for lines like:')
    print('  "Connect via SSH: xxx.localhost.run port 12345"')

In [ ]:
%%capture
# 5. Clone repo
import os
os.chdir('/content')
!rm -rf freqai-futures-strategy
!git clone -q https://github.com/aminak58/freqai-futures-strategy.git
os.chdir('/content/freqai-futures-strategy')
!pip install -q ta-lib-binary 'freqtrade[freqai]'

In [ ]:
# 6. Verify
import os
print(f'✅ Ready: {os.getcwd()}')
!ls -lh | head -10

In [ ]:
# 7. Optional: Mount Drive for data
from google.colab import drive
import zipfile, os
drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/FreqAI/binance_data.zip'
if os.path.exists(zip_path):
    print('📦 Extracting data...')
    os.chdir('/content/freqai-futures-strategy')
    os.makedirs('user_data/data', exist_ok=True)
    with zipfile.ZipFile(zip_path) as z:
        z.extractall('user_data/data/')
    print('✅ Data ready')
else:
    print('ℹ️  No data in Drive')

In [ ]:
# 8. Keep alive
import time
from IPython.display import clear_output
print('🔄 Session active - Tunnel running')
print('='*70)
if tunnel_url:
    print(f'URL: {tunnel_url}:{tunnel_port}')
print(f'Password: {password}')
print('='*70)
print('Updates every 5min\n')
c = 0
while True:
    try:
        time.sleep(300)
        c += 1
        clear_output(wait=True)
        print(f'🔄 Active #{c} ({c*5}min)')
        print('='*70)
        if tunnel_url:
            print(f'URL: {tunnel_url}:{tunnel_port}')
        print(f'Password: {password}')
        print('='*70)
        print(time.strftime('%H:%M:%S'))
    except KeyboardInterrupt:
        break

---
## Troubleshooting

In [ ]:
# Check SSH process
!ps aux | grep ssh

In [ ]:
# Test SSH locally
!ssh -o StrictHostKeyChecking=no root@localhost -p 22 'echo Connection OK'

In [ ]:
# Manual tunnel restart
!pkill -f 'ssh.*localhost.run'
import time
time.sleep(2)
print('Starting fresh tunnel...')
!ssh -R 80:localhost:22 nokey@localhost.run